The main goal is to produce feature dashboards. 

But perhaps what needs to be done along the way is other kinds of analysis: for example, norms of decoder columns (as these SAEs are being trained with Anthropic's April update recipe).

In [1]:
import os
import torch
import plotly.express as px
from tqdm import tqdm
device = "cuda" if torch.cuda.is_available() else "cpu"
from sae_lens.training.session_loader import LMSparseAutoencoderSessionloader
from transformer_lens import utils
from functools import partial

# Set to use 20% of GPU memory, i.e. 8GB on an A100 as ~24 GB is being used to train SAEs
# Can remove this if not training on the GPU at the same time
# if device == "cuda":
#     torch.cuda.set_per_process_memory_fraction(0.2, 0)

torch.set_grad_enabled(False)

# I don't fully understand this but it seems important to avoid some warnings
os.environ["TOKENIZERS_PARALLELISM"] = "false"

# wandb: https://wandb.ai/shehper/gelu-2l-attn-1-sae/reports/gelu-2l-layer-1-attn-heads--Vmlldzo4MDA1NzE4/edit
# expcted losses with SAEs are written in front of the labels below
ckpt_subfolders = { 
    0: "rovi1lwe", #3.785
    1: "p7113j0v", #3.807
    2: "rjc53kjg", #3.768
    3: "hibm6x1l", #3.738
    4: "4xima76s", #3.746
    5: "jq26bfpa", #3.729
    6: "b8e2a9w5", #3.75
    7: "smfws6mc" # 3.748
}

model_name = "gelu-2l"
hook_point_layer=1
hook_point=f"blocks.{hook_point_layer}.attn.hook_z"

d_in= 64
expansion_factor = 32
sae_name = f"{model_name}_{hook_point}_{d_in * expansion_factor}_"

In [2]:
# TODO: Can perhaps probably give load_pretrained_sae a list of paths to load a few different SAEs.

### Code to replace a single head with an SAE

In [3]:
hook_point_head_index = 0 # specify the head index
ckpt_dir = os.path.join("checkpoints", 
                        ckpt_subfolders[hook_point_head_index], 
                        "983044096", # TODO: pick the last ckpt subdir by sorting in
                        sae_name)

model, saes, activations_loader = LMSparseAutoencoderSessionloader.load_pretrained_sae(path=ckpt_dir,
                                                                                        device=device)

# print(saes.autoencoders.keys())
# saes.autoencoders['gelu-2l_blocks.1.attn.hook_z_2048_'].W_dec

sparse_autoencoder = saes.autoencoders[sae_name]
sparse_autoencoder.eval()  # prevents error if we're expecting a dead neuron mask for who grads

Loaded pretrained model gelu-2l into HookedTransformer
Moving model to device:  cpu


Resolving data files:   0%|          | 0/23 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/20 [00:00<?, ?it/s]

SparseAutoencoder(
  (activation_fn): ReLU()
  (hook_sae_in): HookPoint()
  (hook_hidden_pre): HookPoint()
  (hook_hidden_post): HookPoint()
  (hook_sae_out): HookPoint()
)

In [4]:
# activations_loader.store_batch_size_prompts = 16 but that's too large for my macbook
# if we don't specify batch_size in activations_loader.get_batch_tokens() it defaults to this value
# but here I specify it
batch_size = activations_loader.store_batch_size_prompts if device == "cuda" else 1

with torch.no_grad():
    # activation store can give us tokens.
    batch_tokens = activations_loader.get_batch_tokens(batch_size=batch_size) 
    _, cache = model.run_with_cache(batch_tokens, prepend_bos=True)

    # # Use the SAE
    # sae_out, feature_acts, loss, mse_loss, l1_loss, ghost_grad_loss
    sae_out, _, _, _, _, _ = sparse_autoencoder(
        cache[sparse_autoencoder.cfg.hook_point]
    ) 
    


    # # save some room
    # del cache

    # # ignore the bos token, get the number of features that activated in each token, averaged accross batch and position
    # l0 = (feature_acts[:, 1:] > 0).float().sum(-1).detach()
    # print("average l0", l0.mean().item())
    # # px.histogram(l0.flatten().cpu().numpy()).show()

In [5]:
# next we want to do a reconstruction test.
def reconstr_hook(activation, hook, sae_out, hook_point_head_index):
    activation[:, :, hook_point_head_index, :] = sae_out[:, :, hook_point_head_index, :]
    return activation


def zero_abl_hook(activation, hook, hook_point_head_index):
    activation[:, :, hook_point_head_index, :] = torch.zeros(activation.shape[0], activation.shape[1], activation.shape[3])
    # return torch.zeros_like(activation)
    return activation


print("Orig", model(batch_tokens, return_type="loss").item())
print(
    "reconstr",
    model.run_with_hooks(
        batch_tokens,
        fwd_hooks=[
            (
                utils.get_act_name("z", 1), # TODO: is this correct? how do we specify head number?
                partial(reconstr_hook, sae_out=sae_out, hook_point_head_index=hook_point_head_index),
            )
        ],
        return_type="loss",
    ).item(),
)
print(
    "Zero",
    model.run_with_hooks(
        batch_tokens,
        return_type="loss",
        fwd_hooks=[(utils.get_act_name("z", 1), 
                    partial(zero_abl_hook, hook_point_head_index=hook_point_head_index))],
    ).item(),
)

Orig 3.699457883834839
reconstr 3.7783544063568115
Zero 3.8725802898406982


In [6]:
# sparse_autoencoder.W_dec.shape = (2048, 64)
dictionary_norms = torch.linalg.vector_norm(sparse_autoencoder.W_dec, dim=1).cpu().numpy()
print(f"average norm: {dictionary_norms.mean():.4f}")

fig = px.histogram(dictionary_norms, nbins=50, title='Histogram of Dictionary Vector Lengths')
fig.update_layout(
    xaxis_title='Length',
    yaxis_title='Frequency',
    bargap=0.2
)

# Show the plot
fig.show()


average norm: 2.0519


In [7]:
(sparse_autoencoder.W_dec / dictionary_norms[:, None]).shape

torch.Size([2048, 64])

In [8]:
# TODO: does SAELens have code that normalizes weights, biases and activations as in April update?

In [9]:
normalized_dictionary = sparse_autoencoder.W_dec / dictionary_norms[:, None]
normalized_dictionary_norms = torch.linalg.vector_norm(normalized_dictionary, dim=1).cpu().numpy()
print(f"average norm: {normalized_dictionary_norms.mean():.4f}")

fig = px.histogram(normalized_dictionary_norms, nbins=50, title='Histogram of Normalized Dictionary Vector Lengths')
fig.update_layout(
    xaxis_title='Length',
    yaxis_title='Frequency',
    bargap=0.2
)

# Show the plot
fig.show()


average norm: 1.0000


### Let's play around with TransformerLens cache

In [20]:
## The goal is to reproduce "attn_out = W_O @ z".

# batch_tokens = activations_loader.get_batch_tokens(batch_size=batch_size) 
# _, cache = model.run_with_cache(batch_tokens, prepend_bos=True)
hook_z = cache['blocks.1.attn.hook_z'] # (B, T, nh, dh)
B, T, nh, dh = hook_z.shape
hook_attn_out = cache['blocks.1.hook_attn_out'] # (B, T, C)

In [40]:
# model.W_O.shape = (n_layers, nh, dh, n_embd)
# model.b_O.shape = (n_layers, n_embd)
layer1_WO = model.W_O[1] # (nh, dh, n_embd)
layer1_bO = model.b_O[1]
manual_attn_out = torch.einsum("BTNH,NHC->BTC", hook_z, layer1_WO) + layer1_bO
torch.allclose(manual_attn_out, hook_attn_out)

True

In [57]:
cat_hook_z = hook_z.flatten(start_dim=-2, end_dim=-1) # (B, T, C)
cat_WO = layer1_WO.flatten(start_dim=0, end_dim=1) # (C, C)
cat_attn_out = torch.einsum("BTC,CD->BTD", cat_hook_z, cat_WO) + layer1_bO
torch.allclose(cat_attn_out, hook_attn_out, atol=1e-5)

True

In [38]:
layer1_WO = model.W_O[1] # (nh, dh, n_embd)
total = torch.zeros(B, T, nh*dh)
for i in range(nh):
    total += hook_z[:, :, i, :] @ model.W_O[1][i]
total += model.b_O[1]
torch.allclose(total, hook_attn_out, atol=1e-5)

True

In [74]:
head_0_hook_z_lift = torch.cat([hook_z[:, :, 0, :], torch.zeros(B, T, (nh-1)*dh)], dim=-1)
# one can check that head_0_hook_z_lift lies in the dh-dimensional subspace of the concatenated space
# but the action of W_O mixes it with our heads: 
# (head_0_hook_z_lift @ cat_WO) lives in an n_embd-dimensional space and has a projection onto each head's subspace
# print((head_0_hook_z_lift @ cat_WO)[0, 0, :]) 
# can we intepret the projection of one head's output onto another head? This might be easier with feature vectors
# where we can perhaps decompose a feature vector of one had as a linear combination of feature vectors of all heads

In [92]:
# TODO: can we check whether dictionary vectors of each head span the entire head space?
# apparently you can check the rank of each matrix. 
# Use the following as an example. This matrix has rank 3:
# A = torch.tensor([[1, 1, 1],
#                  [3, 2, 1],
#                  [1, 1, 0],
#                  [1, 0, 0]
#                  ], dtype=torch.float32)
# print(torch.linalg.matrix_rank(A))
# TODO: can we check that the dictionary of each head has rank dh?
# If yes, we can just decompose Kissane et al's feature vectors into head feature directions.
# TODO: Is it possible to interpret the decomposition?
print(torch.linalg.matrix_rank(sparse_autoencoder.W_dec).item())

64


In [107]:
# let's take a feature vector of head 0, lift it to the concatenated space and apply W_O.
lifted_feat_vec = torch.cat([sparse_autoencoder.W_dec[0], torch.zeros((nh-1)*dh,)])
(lifted_feat_vec @ cat_WO).shape

# What is the projection of this vector on head 1 space?
(lifted_feat_vec @ cat_WO)[64:128]

# how hard is it to write it as a linear combination of head 1 feature directions?
# in general, it's not going to be unique. But maybe we need to find a sparse decomposition of it?

tensor([-0.0446, -0.0459, -0.1217, -0.1061, -0.0331,  0.1629,  0.1702, -0.1390,
         0.1699, -0.0371, -0.0685,  0.0325,  0.1299,  0.1173,  0.1376, -0.0299,
         0.0787,  0.3107, -0.0155,  0.0416,  0.0492,  0.0835, -0.0504, -0.1963,
        -0.2972,  0.0910,  0.0516,  0.1121, -0.0101, -0.1786, -0.0916, -0.1545,
        -0.0197, -0.1067,  0.0210, -0.2395,  0.0058, -0.0413,  0.0809, -0.0477,
         0.1629, -0.0127,  0.0745, -0.2009, -0.2790, -0.0052, -0.0664, -0.0006,
        -0.0401, -0.1222, -0.0414, -0.0921, -0.0655,  0.0230,  0.1810, -0.0552,
        -0.0809, -0.1069, -0.0894,  0.1107,  0.1168,  0.1726,  0.0180, -0.0350])

In [ ]:
# let's load all SAEs, look at the ranks of dictionaries and see if we can decompose the W_O-output of a feature
# vector into feature vectors from all heads. 

In [ ]:
# TODO: does W_O cause interference between dictionary vectors from different heads?
# take feature vectors from head 0 and feature vectors from head 1. 
# apply W_O to both and calculate the cosine similarities between outputs. 

# TODO: but this has the problem that features don't often fire together. 

In [ ]:
# TODO: how do the reconstructed losses compare if we splice in Kissane et al's SAE vs all of our SAEs?

In [ ]:
# load Kissane et al's SAE, check the rank of its dictionary and decompose their dictionary vectors into our basis. 
# Extra points if we can do it in an interpretable way. 

### Splitting Kissane et al's dictionary into ours

In [115]:
# load all of our dictionaries
n_heads = 8 # gelu-2l has 8 heads
sparse_autoencoders = {}
for hook_point_head_index in range(n_heads):
    ckpt_dir = os.path.join("checkpoints", 
                            ckpt_subfolders[hook_point_head_index], 
                            "983044096", # TODO: pick the last ckpt subdir by sorting in
                            sae_name)

    _, saes, _ = LMSparseAutoencoderSessionloader.load_pretrained_sae(path=ckpt_dir,
                                                                     device=device)

    # print(saes.autoencoders.keys())
    # saes.autoencoders['gelu-2l_blocks.1.attn.hook_z_2048_'].W_dec

    sparse_autoencoder = saes.autoencoders[sae_name]
    sparse_autoencoder.eval()  # prevents error if we're expecting a dead neuron mask for who grads
    sparse_autoencoders[hook_point_head_index] = sparse_autoencoder

Loaded pretrained model gelu-2l into HookedTransformer
Moving model to device:  cpu


Resolving data files:   0%|          | 0/23 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/20 [00:00<?, ?it/s]

Loaded pretrained model gelu-2l into HookedTransformer
Moving model to device:  cpu


Resolving data files:   0%|          | 0/23 [00:00<?, ?it/s]

...

Resolving data files:   0%|          | 0/23 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/20 [00:00<?, ?it/s]

Loaded pretrained model gelu-2l into HookedTransformer
Moving model to device:  cpu


Resolving data files:   0%|          | 0/23 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/20 [00:00<?, ?it/s]

In [129]:
# all heads have rank 64
# for head_id in range(n_heads):
#     print(torch.linalg.matrix_rank(sparse_autoencoders[0].W_dec))

features_per_head, dh = sparse_autoencoders[0].W_dec.shape

# dictionary in concatenated space
cat_dict = torch.zeros(nh*features_per_head, nh*dh)

for head_id in range(n_heads):
    cat_dict[head_id * features_per_head : (head_id + 1) * features_per_head, 
             head_id * dh : (head_id + 1) * dh] = sparse_autoencoders[head_id].W_dec
    
# check that cat_dict has the right rank = n_embd
print(torch.linalg.matrix_rank(cat_dict))

In [130]:
# manually investigate a few features
# maybe instead of generating feature pages for each head individually 
# one can just use the concatenated dict and concatenated feature activations
# this can also be used to perform forward pass through the 

tensor(512)

In [131]:
# TODO: W_O is a mixing matrix between different heads.
# I think that in the circuits paper, they interpret attention heads through W_O * W_V matrix.
# What if we only use W_V to get an interpretation for each head and W_O to get an interpretation 
# for the mixing between heads? 

torch.Size([16384, 512])